# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [ ]:
# imports
import os
import json
import requests
from IPython.display import Markdown, display, update_display
from dotenv import load_dotenv


: 

In [ ]:

# --- Load environment ---
load_dotenv()

MODEL_LLAMA = os.getenv("LOCAL_MODEL_NAME", "llama3.2")
OLLAMA_BASE = os.getenv("OLLAMA_BASE_URL", "http://localhost:11434/v1")

In [ ]:
question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""


In [ ]:


print(f"Getting explanation from {MODEL_LLAMA} using Ollama...")

try:
    response = requests.post(
        f"{OLLAMA_BASE}/completions",
        json={
            "model": MODEL_LLAMA,
            "prompt": question,
            "stream": True
        },
        stream=True,
        timeout=120
    )

    output = ""
    display_handle = display(Markdown("Generating response..."), display_id=True)

    for line in response.iter_lines(decode_unicode=True):
        if not line.strip():
            continue

        # Each event line starts with "data: "
        if line.startswith("data: "):
            line = line[len("data: "):]

        if line.strip() == "[DONE]":
            break

        try:
            data = json.loads(line)
        except json.JSONDecodeError:
            continue

        # In Ollama /v1/completions, the text comes in data['choices'][0]['text']
        text = data.get("choices", [{}])[0].get("text", "")
        if text:
            output += text
            update_display(Markdown(output), display_id=display_handle.display_id)

    print("\nFinal explanation:\n")
    display(Markdown(f"### Llama 3.2 Explanation\n\n{output.strip()}"))

except requests.exceptions.ConnectionError:
    print("Could not connect to Ollama — make sure it’s running (run `ollama serve`).")
except Exception as e:
    print("Unexpected error:", e)
